In [1]:
!pip install openai==0.28 supabase supabase-client python-dotenv langchain[llm]==0.0.228 psycopg2-binary


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00
  Created wheel for supabase-client: filename=supabase_client-0.2.5-py3-none-any

In [1]:
import os
os.chdir('/content/drive/MyDrive/LLM/ChefAssist')

In [5]:
# main.py
from langchain.chat_models import ChatOpenAI
from src.search_recipes import pull_recipes, create_supabase_client
from src.utils import read_config, query_template
from src.run_chains import get_recipe_args, find_recipes
import os
import asyncio

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if __name__ == "__main__":
    # Set up the Supabase client
    #supabase_client = create_supabase_client()

    # Load configuration for API keys
    spoonacular_api_key = read_config()

    OPENAI_KEY =  os.getenv('OPENAI_API_KEY')

    # User query
    user_query = "I want a quick Italian dinner recipe that's vegetarian and includes garlic."
    #user_query = "I want a quick Italian dinner recipe that includes garlic."
    # OpenAI API key - Ensure this is securely stored, e.g., in environment variables or a config file
    OPENAI_KEY = os.getenv("OPENAI_API_KEY")

    # Get recipe search arguments using OpenAI API
    cuisine, dietary_restrictions, meal_type, ingredients, recipe_type = get_recipe_args(user_query)

    # Pull recipes from Spoonacular API and save to Supabase
    db = pull_recipes(
        spoonacular_api_key,
        cuisine,
        dietary_restrictions,
        meal_type,
        ingredients,
        recipe_type,
        10  # Number of results you want
    )

    llm = ChatOpenAI(temperature=0, model="gpt-4-0613", openai_api_key=OPENAI_KEY)
    query = query_template(cuisine, meal_type, dietary_restrictions, ingredients, recipe_type)
    response = find_recipes(query, llm, db)

    print(f"Here's your suggested Recipe: : {response}")




> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: allrecipes, allrecipes_dev, ingredients, recipe, recipe_ingredients, recipes, user_preferences, users, video_semantic_analysis, videos
Thought:The tables that could contain the information I need to answer the question are: `recipes`, `ingredients`, `recipe_ingredients`, and `user_preferences`. I need to check the schema of these tables to understand their structure and the data they contain.

Action: sql_db_schema
Action Input: "recipes, ingredients, recipe_ingredients, user_preferences"
Observation: 
CREATE TABLE ingredients (
	ingredient_id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	CONSTRAINT ingredients_pkey PRIMARY KEY (ingredient_id)
)

/*
3 rows from ingredients table:
ingredient_id	name
170	broccolini
171	garlic clove
172	olive oil
*/


CREATE TABLE recipe_ingredients (
	recipe_id INTEGER NOT NULL, 
	ingredient_id INTEGER NOT NULL, 
	quantity VARCHAR(100), 
	CONSTRAINT recipe_ingredients_

I now know the final answer

Final Answer: Here are some recipes that include "garlic":

1. Recipe ID: 647
   Title: Blended Baked Oats
   Summary: Who says eating your morning oatmeal has to be boring? Try these blended baked oats made with banana, egg, and almond milk. A really ripe banana adds sweetness. It reminds me of a muffin. The flavor possibilities are endless, but I've included my favorites in the footnotes. You can leave out the...
   Ingredients: cooking spray ; 1 cup rolled oats ; 1 large ripe banana ; 1 large egg ; 1 teaspoon baking powder ; â…› teaspoon salt ; Â½ cup almond milk ; 2 tablespoons maple syrup (or sweetener of your choice) ; 1 teaspoon vanilla ; Â½ teaspoon ground cinnamon ; Â¼ cup fresh blueberries
   Directions: Preheat the oven to 350 degrees F (175 degrees C). Spray two 8-ounce ramekins with cooking spray on the bottom and sides. Combine oats, banana, egg, baking powder, salt, almond milk, maple syrup, vanilla, and cinnamon in a blender. Blend until all